In [ ]:
!pip install numpy --upgrade


     |████████████████████████████████| 15.7 MB 255 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


***Misc.***

In [ ]:
# %tensorflow_version 1.x
import tensorflow as tf

In [ ]:
tf.test.gpu_device_name()

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#***Importing Data***




**Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd

**Importing Data**

In [ ]:
def split_data_ml100k(data, num_users, num_items, split_mode="random", test_ratio=0.1):
    """Split the dataset in random mode or seq-aware mode."""
    if split_mode == "seq-aware":
        train_items, test_items, train_list = {}, {}, []
        for line in data.itertuples():
            u, i, rating, time = line[1], line[2], line[3], line[4]
            train_items.setdefault(u, []).append((u, i, rating, time))
            if u not in test_items or test_items[u][-1] < time:
                test_items[u] = (i, rating, time)
        for u in range(1, num_users + 1):
            train_list.extend(sorted(train_items[u], key=lambda k: k[3]))
        test_data = [(key, *value) for key, value in test_items.items()]
        train_data = [item for item in train_list if item not in test_data]
        train_data = pd.DataFrame(train_data)
        test_data = pd.DataFrame(test_data)
    else:
        mask = [True if x == 1 else False for x in np.random.uniform(
            0, 1, (len(data))) < 1 - test_ratio]
        neg_mask = [not x for x in mask]
        train_data, test_data = data[mask], data[neg_mask]
    return train_data, test_data

In [ ]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
item_cols = ['movie id','movie title','release date', 'video release date','IMDb URL','unknown','Action', 'Adventure','Animation','Childrens','Comedy','Crime', 'Documentary','Drama','Fantasy','Film-Noir','Horror', 'Musical','Mystery','Romance ','Sci-Fi','Thriller', 'War' ,'Western']
data = pd.read_csv('./drive/My Drive/Thesis/Soft_Impute/u.data', '\t', names=names, engine='python')  
item = pd.read_csv('./drive/My Drive/Thesis/Soft_Impute/u.item', sep='|', names=item_cols, encoding='latin-1')  

num_users = data.user_id.unique().shape[0]
num_items = data.item_id.unique().shape[0]  

sparsity = 1 - len(data) / (num_users * num_items)
print('number of users: %d, number of items: %d.' % (num_users, num_items))
print('matrix sparsity: %f' % sparsity)
print(data.head(5))


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


number of users: 943, number of items: 1682.
matrix sparsity: 0.936953
   user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596


In [ ]:
train, test = split_data_ml100k(data, num_users, num_items, split_mode="random", test_ratio=0.1)

M_data = data.pivot(index = 'user_id', columns ='item_id', values = 'rating').values
#M_item = item.drop(['movie title','release date', 'video release date','IMDb URL'], axis = 1)
M_data_train = train.pivot(index = 'user_id', columns ='item_id', values = 'rating').values
M_data_test = test.pivot(index = 'user_id', columns ='item_id', values = 'rating').values


Finding location of missing entries

In [ ]:
masked = np.nonzero(pd.isnull(M_data))
xx = masked[0]
yy = masked[1]
missing_mask = np.concatenate((xx[:, None],yy[:, None]), axis=1)

changing nan entries to 0

In [ ]:
M_data = np.nan_to_num(M_data)

Finding location of observed entries

In [ ]:
masked = np.nonzero(M_data)
xx = masked[0]
yy = masked[1]
observed_mask = np.concatenate((xx[:, None],yy[:, None]), axis=1)

Importing 1M dataset

In [ ]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
data = pd.read_csv('./drive/My Drive/Thesis/Soft_Impute/ratings.dat', sep="::", names=names, engine='python', header=None)
M_data = data.pivot(index = 'user_id', columns ='item_id', values = 'rating').values

# **Alternating Proximal**

In [ ]:
import numpy as np
from sklearn.utils.extmath import randomized_svd
import h5py
import numba as nb

In [ ]:
# defining functions
def _max_singular_value(X_filled):
# quick decomposition of X_filled into rank-1 SVD
    _, s, _ = randomized_svd( X_filled, 1, n_iter=5)
    return s[0]

def _converged(X_old, X_new, missing_mask):
    # check for convergence
    old_missing_values = X_old[missing_mask[:, 0], missing_mask[:, 1]]
    new_missing_values = X_new[missing_mask[:, 0], missing_mask[:, 1]]
    difference = old_missing_values - new_missing_values
    ssd = np.sum(difference ** 2)
    old_norm = np.sqrt((old_missing_values ** 2).sum())
    return (np.sqrt(ssd) / old_norm) < convergence_threshold, (np.sqrt(ssd) / old_norm), np.sqrt(ssd)

def masked_mae(X_true, X_pred, mask):
    masked_diff = X_true[mask[:, 0], mask[:, 1]] - X_pred[mask[:, 0], mask[:, 1]]
    return np.mean(np.abs(masked_diff))

In [ ]:
# Execution of loop

shrinkage_value = None
convergence_threshold=0.000001
max_iters = 10000
eta = 0.5

(m, n) = np.shape(M_data)
p = m
A = np.identity(m)
X = np.zeros((m, n))
L = np.zeros((m, n))

obj = np.inf
X_new = []
L_new = []
A_new = []
mae_new = []
obj_new = []
rank_new = []
obj_diff = []
conv_new = []
rmse_new = []

#M_data = M_data/5
M_data = M_data

L0 = M_data
L0_filled = L0
Omega = M_data
norm_Y = np.linalg.norm(L0, ord='fro')**2
tol = 1e-12*norm_Y

X_new.append(X)
A_new.append(A)
L_new.append(L0)


max_singular_value = _max_singular_value(L0)

print("[Alternating Proximal] Max Singular Value of L0 = %f" % (max_singular_value))

shrinkage_value = 10


#if shrinkage_value:
#    shrinkage_value = shrinkage_value
#else:
    # totally hackish heuristic: keep only components
    # with at least 1/50th the max singular value
#    shrinkage_value = max_singular_value / 50.0

[Alternating Proximal] Max Singular Value of L0 = 640.633623


/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


In [ ]:

for i in range(1000):
    obj0 = obj
    L0 = L_new[0]
    Omega = L0
#    X = update_A(n, observed_mask, A, m, shrinkage_value, L0, X)
#        print(j)
    for j in range(n):
        loc_j = np.where(observed_mask[:,1]==j)
        omega_j = observed_mask[loc_j[0], 0]
        Aj = A[omega_j, :]
        I = np.identity(m)
        lamb = shrinkage_value
        yj = L0[omega_j, j]
        Xx = np.dot(np.linalg.inv(np.dot(Aj.T, Aj)+lamb*I), np.dot(Aj.T, yj))
        X[:, j] = Xx
    print("Loop Completed")
    X_new.append(X)
    mu = np.linalg.norm(X, ord=2)
    
    
    temp = Omega*(np.matmul(A, X)-L0)
    grad = np.matmul(temp, X.T)
    rho = (np.linalg.norm(X,2)**2)*eta
    temp = A - grad/rho
    U1,S1, V1 = np.linalg.svd(temp, full_matrices=False, compute_uv=True)
    V1 = V1.T
    diagS = S1
    svp = len(np.asarray(np.where(diagS > shrinkage_value/rho)).T)
    diagS = np.maximum(0, diagS - shrinkage_value/rho)
    rank = (diagS > 0).sum()
    if (svp < 0.5): #svp = 0
        svp = 1
    A = np.matmul(np.matmul(U1[:,:svp], np.diag(diagS[:svp])), V1[:,:svp].T)
    A_new.append(A)
    norm_nu = sum(diagS)
    
    obj = shrinkage_value*norm_nu + 0.5*shrinkage_value*np.linalg.norm(X, ord='fro')**2 + 0.5*np.linalg.norm((np.matmul(A, X)-L0)*Omega, ord='fro')**2
    if (i == 1 or i%1000 == 0 or abs(obj - obj0) < tol):
        print('Iteration # ', iter, ', Objective Value = ', obj, 'rank = ', rank)
    
    
###########################################
        
    L0_reconstruct = np.matmul(A, X)
    L_new.append(L0_reconstruct)

    
  
    converged, conv_val, rmse_val = _converged(X_old=L0_filled, X_new=L0_reconstruct, missing_mask=missing_mask)
    mae = masked_mae(X_true=L0, X_pred=L0_reconstruct, mask=observed_mask)
    L0_filled = L0_reconstruct
    print("[Alternating Proximal] Iter %d: observed rank=%d" % (i + 1, rank))
    print("Sharinkage Value", shrinkage_value)
    print("Mean Absolute Error", mae)
    mae_new.append(mae)
    obj_new.append(obj)
    rank_new.append(rank)
    conv_new.append(conv_val)
    obj_diff.append(obj - obj0)
    rmse_new.append(rmse_val)
    
    print("objective = ",obj)
    print("objective diff = ", obj-obj0)
    
    print("mae = ", mae)
    print("Rank = ", rank)
    print("convolution value = ", conv_val)

    if converged:
        break

    #if (abs(obj - obj0) < tol):
    #    break

print("[Alternating Proximal] Stopped after iteration %d for lambda=%f" % (i + 1, shrinkage_value))
L0_filled[missing_mask[:, 0], missing_mask[:, 1]] = L0_reconstruct[missing_mask[:, 0], missing_mask[:, 1]]




Loop Completed
Iteration #  <built-in function iter> , Objective Value =  67705719.57159477 rank =  943
[Alternating Proximal] Iter 1: observed rank=943
Sharinkage Value 10
Mean Absolute Error 6.0178078167057185
objective =  67705719.57159477
objective diff =  -inf
mae =  6.0178078167057185
Rank =  943
convolution value =  inf


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  


Loop Completed
Iteration #  <built-in function iter> , Objective Value =  2340548.697196615 rank =  943
[Alternating Proximal] Iter 2: observed rank=943
Sharinkage Value 10
Mean Absolute Error 1.1146704944919097
objective =  2340548.697196615
objective diff =  -65365170.87439816
mae =  1.1146704944919097
Rank =  943
convolution value =  0.7214985999022951
Loop Completed
[Alternating Proximal] Iter 3: observed rank=943
Sharinkage Value 10
Mean Absolute Error 0.7038484699230021
objective =  778314.6928979288
objective diff =  -1562234.004298686
mae =  0.7038484699230021
Rank =  943
convolution value =  0.2784935597391817
Loop Completed
[Alternating Proximal] Iter 4: observed rank=943
Sharinkage Value 10
Mean Absolute Error 0.6265389431422375
objective =  419232.9485460007
objective diff =  -359081.74435192806
mae =  0.6265389431422375
Rank =  943
convolution value =  0.13378968017188692
Loop Completed
[Alternating Proximal] Iter 5: observed rank=943
Sharinkage Value 10
Mean Absolute Erro

In [ ]:
xx = pd.DataFrame(conv_new)
xx.to_csv('/content/drive/MyDrive/conv_val_AP.csv')
xx = pd.DataFrame(rmse_new)
xx.to_csv('/content/drive/MyDrive/rmse_val_AP.csv')

In [ ]:
import matplotlib.pyplot as plt
title_font = {'fontname':'Arial', 'size':'20', 'color':'black', 'weight':'normal',
              'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'Arial', 'size':'20'}

plt.figure()
plt.semilogx(conv_new)
plt.xlabel("Iteration Number", **axis_font)
#plt.ylabel("Convergence Value", **axis_font)
plt.title("Relative Error", y = 0.85, **title_font)
plt.rc('xtick', labelsize=15)    # fontsize of the tick labels
plt.rc('ytick', labelsize=15)
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/AP_relative_error.png')

plt.figure()
plt.semilogx(rmse_new)
plt.xlabel("Iteration Number", **axis_font)
#plt.ylabel("Convergence Value", **axis_font)
plt.title("RMSE", y = 0.85, **title_font)
plt.rc('xtick', labelsize=15)    # fontsize of the tick labels
plt.rc('ytick', labelsize=15)
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/AP_rmse.png')

In [ ]:
import math as m

X_true=L_new[0]
X_pred=L0_reconstruct
mask=observed_mask

old_missing_values = X_true[mask[:, 0], mask[:, 1]]
new_missing_values = X_pred[mask[:, 0], mask[:, 1]]
difference = old_missing_values - new_missing_values
ssd = np.sum(difference ** 2)

AIC = 2*rank*np.shape(X_true)[0]+len(difference)*m.log(ssd)
BIC = np.log(len(difference))*rank*np.shape(X_true)[0]+len(difference)*m.log(ssd)

print(AIC)
print(BIC)
print(1.0 - np.count_nonzero(X_pred) / X_pred.size)

# **Misc**

In [ ]:
print("[Alternating Proximal] Stopped after iteration %d for lambda=%f" % (i + 1, shrinkage_value))
L0_filled[observed_mask[:, 0], observed_mask[:, 1]] = L0[observed_mask[:, 0], observed_mask[:, 1]]   
L0_filled[missing_mask[:, 0], missing_mask[:, 1]] = L0_reconstruct[missing_mask[:, 0], missing_mask[:, 1]]   

h5f_data = h5py.File('./drive/My Drive/Thesis/L0_filled.h5', 'w')
h5f_data.create_dataset('L0_filled', data=L0_filled)
h5f_data.close()

h5f_data = h5py.File('./drive/My Drive/Thesis/mae_new.h5', 'w')
h5f_data.create_dataset('mae_new', data=mae_new)
h5f_data.close()

h5f_data = h5py.File('./drive/My Drive/Thesis/rank_new.h5', 'w')
h5f_data.create_dataset('rank_new', data=rank_new)
h5f_data.close()

h5f_data = h5py.File('./drive/My Drive/Thesis/obj_new.h5', 'w')
h5f_data.create_dataset('obj_new', data=obj_new)
h5f_data.close()

h5f_data = h5py.File('./drive/My Drive/Thesis/obj_diff.h5', 'w')
h5f_data.create_dataset('obj_diff', data=obj_diff)
h5f_data.close()

h5f_data = h5py.File('./drive/My Drive/Thesis/A_new.h5', 'w')
h5f_data.create_dataset('A_new', data=A_new)
h5f_data.close()

h5f_data = h5py.File('./drive/My Drive/Thesis/X_new.h5', 'w')
h5f_data.create_dataset('X_new', data=X_new)
h5f_data.close()

h5f_data = h5py.File('./drive/My Drive/Thesis/L0_new.h5', 'w')
h5f_data.create_dataset('L0_new', data=L0_new)
h5f_data.close()

In [ ]:
 h5f_data = h5py.File('./drive/My Drive/Thesis/obj_diff.h5', 'w')
h5f_data.create_dataset('obj_diff', data=obj_diff)
h5f_data.close()

In [ ]:
h5f_data.close()

In [ ]:
h5f_data = h5py.File('./drive/My Drive/Thesis/A_new.h5', 'w')
h5f_data.create_dataset('A_new', data=A_new)
h5f_data.close()

h5f_data = h5py.File('./drive/My Drive/Thesis/X_new.h5', 'w')
h5f_data.create_dataset('X_new', data=X_new)
h5f_data.close()